# Run constrained emissions-driven ensemble in SSP2-4.5

-  verify forcings are close to assessment

In [1]:
import os.path

import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyam
from fair.forward import fair_scm
from scmdata import ScmRun, run_append
from tqdm import tqdm_notebook

import openscm_runner
from openscm_runner.run import run
from openscm_runner.adapters import FAIR

<IPython.core.display.Javascript object>

/nfs/b0110/Users/mencsm/miniconda3/envs/ar6/lib/python3.7/site-packages/openscm_runner/run.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
openscm_runner.__version__

'0.4.4'

In [3]:
fair = FAIR()
fair.get_version()

'1.6.2'

In [4]:
with open('../data_output_large/fair-samples/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

In [5]:
# what a pain

species = [
'Emissions|BC',
'Emissions|CH4',
'Emissions|CO',
'Emissions|CO2|MAGICC AFOLU',
'Emissions|CO2|MAGICC Fossil and Industrial',
'Emissions|F-Gases|HFC|HFC125',
'Emissions|F-Gases|HFC|HFC134a',
'Emissions|F-Gases|HFC|HFC143a',
'Emissions|F-Gases|HFC|HFC227ea',
'Emissions|F-Gases|HFC|HFC23',
'Emissions|F-Gases|HFC|HFC245fa',
'Emissions|F-Gases|HFC|HFC32',
'Emissions|F-Gases|HFC|HFC4310mee',
'Emissions|F-Gases|PFC|C2F6',
'Emissions|F-Gases|PFC|C6F14',
'Emissions|F-Gases|PFC|CF4',
'Emissions|F-Gases|SF6',
'Emissions|Montreal Gases|CCl4',
'Emissions|Montreal Gases|CFC|CFC11',
'Emissions|Montreal Gases|CFC|CFC113',
'Emissions|Montreal Gases|CFC|CFC114',
'Emissions|Montreal Gases|CFC|CFC115',
'Emissions|Montreal Gases|CFC|CFC12',
'Emissions|Montreal Gases|CH3Br',
'Emissions|Montreal Gases|CH3CCl3',
'Emissions|Montreal Gases|CH3Cl',
'Emissions|Montreal Gases|HCFC141b',
'Emissions|Montreal Gases|HCFC142b',
'Emissions|Montreal Gases|HCFC22',
'Emissions|Montreal Gases|Halon1202',
'Emissions|Montreal Gases|Halon1211',
'Emissions|Montreal Gases|Halon1301',
'Emissions|Montreal Gases|Halon2402',
'Emissions|N2O',
'Emissions|NH3',
'Emissions|NOx',
'Emissions|OC',
'Emissions|Sulfur',
'Emissions|VOC']

In [6]:
df_fair = ScmRun('../data_input_large/rcmip-emissions-annual-means-v5-1-0.csv', lowercase_cols=True)
df_fair.filter(
    scenario=['ssp245'], 
#    year=range(2015,2301),
    year=range(2015,2101),
    variable=species,
    region='World', 
    inplace=True
)
print(len(df_fair))
#df_fair.head(50)

39


In [7]:
nt = df_fair.time_points.years()[-1] - 1750 + 1
nt

351

In [8]:
# convert lists to numpy arrays
updated_config = []
for i in range(len(config_list)):
    updated_config.append({})
    for key, value in config_list[i].items():
        if type(value)==list:
            updated_config[i][key] = np.asarray(value)
        else:
            updated_config[i][key] = value

In [9]:
# need parallel FaIR in openscm-runner

x = run(
    climate_models_cfgs={
        "FAIR": updated_config,
    },
    scenarios=df_fair,
    output_variables=(
        "Surface Temperature",
        "Surface Temperature (GMST)",
        "Atmospheric Concentrations|CO2",
        "Atmospheric Concentrations|CH4",
        "Atmospheric Concentrations|N2O",
        "Effective Radiative Forcing",
        "Effective Radiative Forcing|CO2",
        "Effective Radiative Forcing|CH4",
        "Effective Radiative Forcing|N2O",
        "Effective Radiative Forcing|Greenhouse Gases",
        "Effective Radiative Forcing|Tropospheric Ozone",
        "Effective Radiative Forcing|CH4 Oxidation Stratospheric H2O",
        "Effective Radiative Forcing|Contrails",
        "Effective Radiative Forcing|Aerosols",
        "Effective Radiative Forcing|Aerosols|Direct Effect|BC",
        "Effective Radiative Forcing|Aerosols|Direct Effect|OC",
        "Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur",
        "Effective Radiative Forcing|Aerosols|Direct Effect|Nitrate",
        "Effective Radiative Forcing|Aerosols|Direct Effect",
        "Effective Radiative Forcing|Aerosols|Indirect Effect",
        "Effective Radiative Forcing|Black Carbon on Snow",
        "Effective Radiative Forcing|Land-use Change"
    ),
)

Climate models:   0%|          | 0/1 [00:00<?, ?it/s]

Creating FaIR emissions inputs:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# convert to ScmRun for better plotting functionality
x = ScmRun(x.timeseries())

In [11]:
x.tail()

time                                                                                                              1750-01-01  \
climate_model model           region run_id scenario unit   variable                                                           
FaIRv1.6.2    MESSAGE-GLOBIOM World  2048   ssp245   W/m**2 Effective Radiative Forcing|Aerosols|Direct Eff... -1.933205e-12   
                                                            Effective Radiative Forcing|Aerosols|Direct Effect  5.735094e-10   
                                                            Effective Radiative Forcing|Aerosols|Indirect E...  5.432287e-10   
                                                            Effective Radiative Forcing|Black Carbon on Snow    7.396161e-11   
                                                            Effective Radiative Forcing|Land-use Change        -9.371003e-05   

time                                                                                                            1751-01-01  \
climate_model model           region run_id scenario unit   variable                                                         
FaIRv1.6.2    MESSAGE-GLOBIOM World  2048   ssp245   W/m**2 Effective Radiative Forcing|Aerosols|Direct Eff...    0.000131   
                                                            Effective Radiative Forcing|Aerosols|Direct Effect   -0.000629   
                                                            Effective Radiative Forcing|Aerosols|Indirect E...    0.003442   
                                                            Effective Radiative Forcing|Black Carbon on Snow     -0.000368   
                                                            Effective Radiative Forcing|Land-use Change          -0.000189   

time                                                                                                            1752-01-01  \
climate_model model           region run_id scenario unit   variable                                                         
FaIRv1.6.2    MESSAGE-GLOBIOM World  2048   ssp245   W/m**2 Effective Radiative Forcing|Aerosols|Direct Eff...    0.000209   
                                                            Effective Radiative Forcing|Aerosols|Direct Effect   -0.000568   
                                                            Effective Radiative Forcing|Aerosols|Indirect E...    0.005300   
                                                            Effective Radiative Forcing|Black Carbon on Snow     -0.000495   
                                                            Effective Radiative Forcing|Land-use Change          -0.000286   

time                                                                                                            1753-01-01  \
climate_model model           region run_id scenario unit   variable                                                         
FaIRv1.6.2    MESSAGE-GLOBIOM World  2048   ssp245   W/m**2 Effective Radiative Forcing|Aerosols|Direct Eff...    0.000089   
                                                            Effective Radiative Forcing|Aerosols|Direct Effect   -0.000726   
                                                            Effective Radiative Forcing|Aerosols|Indirect E...    0.003471   
                                                            Effective Radiative Forcing|Black Carbon on Snow     -0.000382   
                                                            Effective Radiative Forcing|Land-use Change          -0.000385   

time                                                                                                            1754-01-01  \
climate_model model           region run_id scenario unit   variable                                                         
FaIRv1.6.2    MESSAGE-GLOBIOM World  2048   ssp245   W/m**2 Effective Radiative Forcing|Aerosols|Direct Eff...   -0.000169   
                                                            Effective Radiat

In [12]:
# co2 409.9
np.percentile(x.filter(variable="Atmospheric Concentrations|CO2", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([409.0799415 , 410.13768532, 411.21770688])

In [13]:
# ch4 1866.3
np.percentile(x.filter(variable="Atmospheric Concentrations|CH4", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([1868.81582612, 1868.81582612, 1868.81582612])

In [14]:
# n2o 332.1
np.percentile(x.filter(variable="Atmospheric Concentrations|N2O", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([332.32989281, 332.32989281, 332.32989281])

In [15]:
# co2 1.90 2.16 2.41
# any differences to GHG forcing is as likely to be with the pre-industrial concentration as it is with FaIR itself
np.percentile(x.filter(variable="Effective Radiative Forcing|CO2", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([1.93389225, 2.21294904, 2.48878616])

In [16]:
# ch4 0.43 0.54 0.65
np.percentile(x.filter(variable="Effective Radiative Forcing|CH4", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([0.43320942, 0.5471859 , 0.65354015])

In [17]:
# n2o 0.18 0.21 0.24
np.percentile(x.filter(variable="Effective Radiative Forcing|N2O", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([0.16852683, 0.19654704, 0.22267794])

In [18]:
# other wmghg 0.33 0.41 0.49
np.percentile(
    (
        x.filter(variable="Effective Radiative Forcing|Greenhouse Gases", scenario='ssp245', year=2019).timeseries().values.squeeze() - 
        x.filter(variable="Effective Radiative Forcing|CO2", scenario='ssp245', year=2019).timeseries().values.squeeze() -
        x.filter(variable="Effective Radiative Forcing|CH4", scenario='ssp245', year=2019).timeseries().values.squeeze() -
        x.filter(variable="Effective Radiative Forcing|N2O", scenario='ssp245', year=2019).timeseries().values.squeeze()
    )
, (5, 50, 95))

array([0.30891275, 0.38111656, 0.45100342])

In [19]:
x.filter(variable="Effective Radiative Forcing|N2O", scenario='ssp245', year=2019).timeseries()

time                                                                                         2019-01-01
climate_model model           region run_id scenario unit   variable                                   
FaIRv1.6.2    MESSAGE-GLOBIOM World  0      ssp245   W/m**2 Effective Radiative Forcing|N2O    0.231808
                                     1      ssp245   W/m**2 Effective Radiative Forcing|N2O    0.194334
                                     2      ssp245   W/m**2 Effective Radiative Forcing|N2O    0.218864
                                     3      ssp245   W/m**2 Effective Radiative Forcing|N2O    0.206190
                                     4      ssp245   W/m**2 Effective Radiative Forcing|N2O    0.195785
...                                                                                                 ...
                                     2044   ssp245   W/m**2 Effective Radiative Forcing|N2O    0.222928
                                     2045   ssp245   W/m**2 Effective Radiative Forcing|N2O    0.202872
                                     2046   ssp245   W/m**2 Effective Radiative Forcing|N2O    0.203787
                                     2047   ssp245   W/m**2 Effective Radiative Forcing|N2O    0.190044
                                     2048   ssp245   W/m**2 Effective Radiative Forcing|N2O    0.202290

[2049 rows x 1 columns]

In [20]:
# n2o 0.18 0.21 0.24
np.percentile(x.filter(variable="Effective Radiative Forcing|N2O", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([0.16852683, 0.19654704, 0.22267794])

In [21]:
# o3 0.24 0.47 0.71
np.percentile(x.filter(variable="Effective Radiative Forcing|Tropospheric Ozone", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([0.21505865, 0.44448128, 0.67491947])

In [22]:
# h2o 0 0.05 0.10
np.percentile(x.filter(variable="Effective Radiative Forcing|CH4 Oxidation Stratospheric H2O", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([-0.00189996,  0.04990273,  0.10009878])

In [23]:
# ERFaer -0.6 -0.3 -0.0
np.percentile(x.filter(variable="Effective Radiative Forcing|Aerosols|Direct Effect", scenario='ssp245', year=np.arange(2005,2015)).timeseries().values.squeeze(), (5, 50, 95))

array([-0.56769848, -0.27577097,  0.00135659])

In [24]:
# ERFaci -1.7 -1.0 -0.3   - very hard to get
np.percentile(x.filter(variable="Effective Radiative Forcing|Aerosols|Indirect Effect", scenario='ssp245', year=np.arange(2005,2015)).timeseries().values.squeeze(), (5, 50, 95))

array([-1.41563948, -0.85284098, -0.32777979])

In [25]:
# BC Snow 0.02 0.08 0.18
np.percentile(x.filter(variable="Effective Radiative Forcing|Black Carbon on Snow", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([0.02062592, 0.09115713, 0.20588791])

In [26]:
# Land use -0.30 -0.20 -0.10
np.percentile(x.filter(variable="Effective Radiative Forcing|Land-use Change", scenario='ssp245', year=2019).timeseries().values.squeeze(), (5, 50, 95))

array([-0.29814782, -0.19945219, -0.10227193])

In [27]:
config_list[0]['scale']

[1.0,
 0.9145866648321018,
 1.2661237500125642,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.1582002705100407,
 1.147950710594023,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.024955991601806,
 1.0,
 1.0,
 1.4468188139272717,
 1.0283056135367221,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9140050659337687,
 1.43808811525497,
 1.0360710019580495,
 1.0]

In [28]:
scale_normals = np.load('../data_input_large/fair-samples/scale_normals.npy')
scale_normals[838]

array([0.91283067, 1.043585  , 1.08659637, 0.88729184, 1.36324128,
       0.0405751 , 1.22167194, 1.4286079 , 1.12050869, 1.12097391,
       0.5710494 ])